In [1]:
#!pip install git+https://github.com/openai/whisper.git

# ivrit

In [2]:
# https://medium.com/@DormanDaniel/comparing-whisper-whisper-ft-and-amazon-transcribe-for-hebrew-e297846bdd24
#https://github.com/yairl/ivrit.ai/blob/master/run_whisper.py

In [3]:
model_nm = 'ivrit-ai/whisper-13-v2-e3'
model_nm = "ivrit-ai/whisper-large-v2-tuned"

In [4]:
audio_file_path = "/kaggle/input/speech-to-text/harvard.wav"
audio_file_path = "/kaggle/input/speech-to-text/HEB_M_OferS.mp3"
audio_file_path = "/kaggle/input/speech-to-text/sample4.mp3"


In [5]:
import torch
from transformers import pipeline


device = "cuda:0" if torch.cuda.is_available() else "cpu"

pipe = pipeline(
  "automatic-speech-recognition",
  model= model_nm,
  chunk_length_s=30,
  batch_size=8,
  device=device,
)


2024-07-04 17:36:10.985032: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-04 17:36:10.985103: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-04 17:36:10.986794: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
%%time
prediction = pipe(audio_file_path, generate_kwargs={"language": "hebrew"})["text"]

CPU times: user 14 s, sys: 249 ms, total: 14.3 s
Wall time: 14.5 s


In [12]:
prediction

"זה חברת רות גלעד, אני יודע שאתה לא אוהב אותנו אז אני אתאמציתי וקצר אני מתקשר סך הכל לגבי העובדה שאתה לקוח שלנו ועזבת אותנובשמחההייתי מסיים את הארבע שעות האלה הייתי צריך לעלות לחדרים שעשו צ'קאוט ולפתוח את הכספות לוודא שהם לא השאירו אותם נאולות אחת מתוך כל שמונה כספות הייתי בה הייתי מוצא חיתול עם קקי של תינוק בתוך הכספת אוקיי? ואת כל הדבר הזה אני עשיתי בשביל שכר מינימום אני מעדיף לחזור עכשיו, עכשיו תשגר אותי לשם בטלפורטציה, לפתוח כספות עם קקי ולעמוד בכניסה למלון בזמן שאיזה קוריאני מנפיח את נשמתו על הפרצוף שלי מה שיחזור להוט וגם כדי יש מחיר טוב כולל טלוויזיה אינטרנט וטלפון"

# faster whisper

In [8]:
! pip install ffmpeg
! pip install faster_whisper
import ffmpeg
import faster_whisper
import time

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()


In [13]:
# Initialize the model
model = faster_whisper.WhisperModel('ivrit-ai/faster-whisper-v2-pd1-e1')

RuntimeError: CUDA failed with error out of memory

In [ ]:
def transcribe_file(file_name):
    start = time.time()
    
    segs, _ = model.transcribe(file_name, language='he')
    l = []
    for seg in segs:
        l.append(seg.text)
    end = time.time()
    run_time = end - start
    
    return "".join(l),run_time

In [14]:
import pandas as pd

In [15]:
file1 = '/kaggle/input/speech-to-text/sample4_out_1.mp3'
file2 = '/kaggle/input/speech-to-text/sample4_out_2.mp3'
file3 = '/kaggle/input/speech-to-text/sample4_out_3.mp3'
file4 = '/kaggle/input/speech-to-text/sample4_out_4.mp3'


In [16]:
file1 ='/kaggle/input/speech-to-text/sample1.mp3'
file2 ='/kaggle/input/speech-to-text/sample2.mp3'
file3 ='/kaggle/input/speech-to-text/sample3.mp3'
file4 ='/kaggle/input/speech-to-text/sample4.mp3'

file6 ='/kaggle/input/speech-to-text/smaple6.mp3'

In [ ]:
files = [file1,file2,file3,file4,file6]
d = []
for i,f in enumerate(files):
    print(i, end='')
    
    text,run_time = transcribe_file(f)
    
    d.append(
        {
            'name':f,
            'text':text,
            'run_time':run_time,
            'engine':'GPUT4*2'
  
                    }
    )

results = pd.DataFrame(d)
results

In [24]:
#results.to_excel('cpubench.xlsx')
#results.to_excel('T4bench.xlsx')
#results.to_excel('T4bench.xlsx')
results = pd.read_excel('T4bench.xlsx',index_col=[0])
results

,name,text,run_time,engine
0,/kaggle/input/speech-to-text/sample1.mp3,"אנא הג'נד, סופר. יפה מאוד, יפה מאוד. עוד עליך,...",31.665811,GPUT4*2
1,/kaggle/input/speech-to-text/sample2.mp3,"היי, רותי. רועי מר-אל מהביטוח שלכם. מה שלומך? ...",16.178251,GPUT4*2
2,/kaggle/input/speech-to-text/sample3.mp3,"שלום. ותודה שאת ניתן לקרוא. מקומכם בתור הינו, ...",19.817106,GPUT4*2
3,/kaggle/input/speech-to-text/sample4.mp3,"זו חברת הוט גלעד, אני יודע שאתה לא אוהב אותנו ...",18.442701,GPUT4*2
4,/kaggle/input/speech-to-text/smaple6.mp3,"אוקיי, ברוכים הבאים לקורס תמיכה טכנית, הקורס ה...",262.721325,GPUT4*2


In [25]:
results.iloc[3]['text']

"זו חברת הוט גלעד, אני יודע שאתה לא אוהב אותנו וזה נהיה תמציתי וקצר. אני מתקשר סך הכול לגבי העובדה שהיא הייתה לקוח שלנו ועזבת אותנו. תקשיב, אני דיברתי איתכם כבר איזה 9 פעמים ואתם לא מפסיקים להתקשר אליי. אני לא יודע מה עוד אני יכול לעשות. אני כאילו... לא, לא, זה... מה יכול להיות שתירש? אני רוצה לספר לך סיפור, סבבה? אני רוצה לספר לך סיפור. תקשיב שנייה. בשמחה. אני, אחרי הצבא, הייתי צריך כסף, נכון? אז אני הלכתי למצוא לי עבודה מועדפת. אני הלכתי להיות קבט במלון בירושלים, סבבה? אני הייתי עומד בכניסה למלון, סבבה? עם מגנומטר. התיירים היו נכנסים ויוצאים מהמלון. עכשיו, הרבה מהתיירים האלה היו קוריאנים. עכשיו, קוריאנים, אני לא יודע אם אתה יודע, בתרבות שלהם אין דבר כזה לשמור ופלוצים בבטן. הם מאמינים בלשחרר. אז דמיין לעצמך. אני הייתי עומד בכניסה למלון, וקוריאנים פשוט מפליצים עליי, בלי הפסקה. מאשנים סיגרות ומפליצים. הייתי מסיים את הארבע השעות האלה, הייתי צריך להעלות לחדרים שעשו צ'קאוט, ולפתוח את הכספות, לוודא שהם לא השאירו אותן נאולות. אחת מתוך כל שמונה כספות הייתי באה, הייתי מוצא חיתול עם קקי של תינו

In [21]:
prediction

"זה חברת רות גלעד, אני יודע שאתה לא אוהב אותנו אז אני אתאמציתי וקצר אני מתקשר סך הכל לגבי העובדה שאתה לקוח שלנו ועזבת אותנובשמחההייתי מסיים את הארבע שעות האלה הייתי צריך לעלות לחדרים שעשו צ'קאוט ולפתוח את הכספות לוודא שהם לא השאירו אותם נאולות אחת מתוך כל שמונה כספות הייתי בה הייתי מוצא חיתול עם קקי של תינוק בתוך הכספת אוקיי? ואת כל הדבר הזה אני עשיתי בשביל שכר מינימום אני מעדיף לחזור עכשיו, עכשיו תשגר אותי לשם בטלפורטציה, לפתוח כספות עם קקי ולעמוד בכניסה למלון בזמן שאיזה קוריאני מנפיח את נשמתו על הפרצוף שלי מה שיחזור להוט וגם כדי יש מחיר טוב כולל טלוויזיה אינטרנט וטלפון"

In [26]:
results.iloc[4]['text']

'אוקיי, ברוכים הבאים לקורס תמיכה טכנית, הקורס הפרקטי הראשון לתמיכה טכנית, שאפשר להגיד את המקבלים, הכל מול העיניים, פה אין מצגת, אני מראה לכם ממש בצורה הכי פרקטית, הכי מהירה שיש, איך מטפלים במחשב ואיך מזהים בעיות.אוקיי? עכשיו, מי שאהב את הקורס של הטכנאי-פי-סי ורוצה דרך ב\', או דרך ג\', שדרך אגב, דרך ג\' זה אלקטרוניקה, אבל מי שרוצה כמובן דרך ב\', דרך ג\', אני מוכן בכיף, באהבה, רק תרשמו לי תגובה, או שתשלחו לי הודעה לטלפון, ואני אדאג לזה. אוקיי? אז בואו נתחיל, בואו נתחיל בנושא הראשון, משאבי מערכת ותהליכים. אוקיי? אז בואו, בשורת משימות, נלחץ לחצן ימני ונלחץ task manager, מנהל המשימות, בעברית. אוקיי? פה אתם רואים שיש לי פה את כל התוכנות שרצות, ופה בצד אני יכול לראות כמה זה לוקח מהזיכרון. כרגע יש לי דפוסה של 61 מהזיכרון, ויש לי 5 אחוז CPU. בואו ננסה לדנקות קצת, בואו ננקה את ג\'יגה, נעשה פה end task, ופתאום זה ירד ל-54. אוקיי? פשוט מאוד. המחשב עובד לאט, זה אחד, זה יכול להיות אחד הסיבות המרכזיות לדברים שקוראים. והנה, יש לכם פה מבט מעוף יותר, יותר טוב. אתם יכולים גם לראות כמה ה-hard disk שלכם עו

In [ ]:
#pd.concat([a,b],ignore_index=True)

In [ ]:
# parameters optimizations
# todo other models
# other engines gpus

# get sentiment